In [ ]:
import numpy as np
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append("..")
from utils import *
import warnings
warnings.filterwarnings("ignore")
import os
import psutil

pd.options.display.max_columns=1000
pd.options.display.max_rows=1000

import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score 
import optuna
from optuna import Trial
from optuna.samplers import TPESampler


In [ ]:
''' Seed '''
seed_everything(42)

''' Data Load '''
train, test, sample_submission = load_data()

''' Train / Vaild Split'''
X_train = train.drop(columns = ["Click"]) 
y_train = train["Click"]
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

''' preprocessing '''
train_fre, test_fre = preprocessing(X_train, X_valid, test) 

In [ ]:
def objective(trial, t_X, t_y, v_X, v_y):
  param = {"n_estimators": trial.suggest_int("n_estimators:", 100, 1000),
           "max_depth": trial.suggest_int("max_depth", 6, 30),
           "subsample": trial.suggest_float("subsample", 0.3, 1.0),
           "learning_rate":  trial.suggest_float("learning_rate", 0.01, 0.3),
           'lambda': trial.suggest_float('lambda', 1e-3, 0.1),
           'alpha': trial.suggest_float('alpha', 1e-3, 1.0),
           'min_child_weight': trial.suggest_int('min_child_weight', 2, 50)}

  model = lgb.LGBMClassifier(random_state=42, **param)

  model.fit(t_X, t_y, eval_metric='AUC')
  pred = model.predict_proba(v_X)
  score = roc_auc_score(v_y, pred[:, 1])

  return score

In [ ]:
study = optuna.create_study(study_name='XCClassifier', direction='maximize', sampler=TPESampler(seed=42))
study.optimize(lambda trial: objective(trial, X_train, y_train, X_valid, y_valid), n_trials=20)

print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)